In [0]:
#8	ResellerSales	Currency.parquet	stg.currency	abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/Sales.Currency/	Bronze	RS_Bronze_01	ResellerSales_Bronze	stg.currency	abfss://project@scrgvkrmade.dfs.core.windows.net/silver/stg_currency/	Databricks/Transformation	Parallel	Blocked	Currency.parquet (Raw Bronze)	Silver	stg.currency	ModifiedDate	["CurrencyCode","CurrencyAlternateKey","CurrencyName","ModifiedDate","__ingest_ts","__source_file","__source_path","__batch_id","__row_hash"]	true	false	["CurrencyAlternateKey"]	2025-12-02 07:40:46.9809009
# Source_path : abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/Sales.Currency/
# Target_path : abfss://project@scrgvkrmade.dfs.core.windows.net/silver/stg_currency/
# column_list : ["CurrencyCode","CurrencyAlternateKey","CurrencyName","ModifiedDate","__ingest_ts","__source_file","__source_path","__batch_id","__row_hash"]
# direct_account_key : E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg==
# domain: ResellerSales
# file_name: stg.currency
# include_layer :false
# layer:Bronze
# table_name:stg.currency
# year_column:ModifiedDate
# incremental_flag : false
# merge_flag : true
# pk_columns : CurrencyAlternateKey
# 04_BronzeToSilver
# Purpose: read bronze parquet (recursive), clean/transform, and MERGE into silver Delta.
# Save this as a Databricks Python notebook (e.g. /Repos/.../04_BronzeToSilver)

import sys, importlib, json, traceback, uuid
from pyspark.sql.functions import col, trim, to_timestamp, year, lit, concat_ws, sha2, current_timestamp
from pyspark.sql import SparkSession

# Optional utils module if present
if "/dbfs/FileStore" not in sys.path:
    sys.path.insert(0, "/dbfs/FileStore")
try:
    import utils_etl
    importlib.reload(utils_etl)
    print("utils_etl imported (optional).")
except Exception:
    pass

# ---------------------------
# WIDGETS / DEFAULT PARAMETERS
# (ADF will override these at runtime — defaults set to the values you provided)
# ---------------------------
dbutils.widgets.text("Source_path", "abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/Sales.Currency/")
dbutils.widgets.text("Target_path", "abfss://project@scrgvkrmade.dfs.core.windows.net/silver/stg_currency/")
dbutils.widgets.text("column_list", '["CurrencyCode","CurrencyAlternateKey","CurrencyName","ModifiedDate","__ingest_ts","__source_file","__source_path","__batch_id","__row_hash"]')
dbutils.widgets.text("direct_account_key", "E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg==")
dbutils.widgets.text("domain", "ResellerSales")
dbutils.widgets.text("file_name", "stg.currency")
dbutils.widgets.text("include_layer", "false")
dbutils.widgets.text("layer", "Bronze")
dbutils.widgets.text("table_name", "stg.currency")
dbutils.widgets.text("year_column", "ModifiedDate")
dbutils.widgets.text("incremental_flag", "false")
dbutils.widgets.text("merge_flag", "true")
dbutils.widgets.text("pk_columns", '["CurrencyAlternateKey"]')
dbutils.widgets.text("watermark", "")   # optional; not used since incremental_flag=false
dbutils.widgets.text("batch_name", "Batch B")  # optional audit

# ---------------------------
# Read widget values
# ---------------------------
params = {k: dbutils.widgets.get(k).strip() for k in [
    "Source_path","Target_path","column_list","direct_account_key","domain","file_name","include_layer",
    "layer","table_name","year_column","incremental_flag","merge_flag","pk_columns","watermark","batch_name"
]}

print("Parameters (summary):")
print(" source:", params["Source_path"])
print(" target:", params["Target_path"])
print(" table_name:", params["table_name"])
print(" merge_flag:", params["merge_flag"], " incremental_flag:", params["incremental_flag"])
print(" pk_columns:", params["pk_columns"])
print(" column_list:", params["column_list"])

# ---------------------------
# Helpers
# ---------------------------
def parse_list(txt):
    if not txt:
        return []
    s = txt.strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            arr = json.loads(s)
            return [str(x).strip() for x in arr]
        except Exception:
            # fallback
            s2 = s.strip("[]")
            return [p.strip().strip('"').strip("'") for p in s2.split(",") if p.strip()]
    return [p.strip() for p in s.split(",") if p.strip()]

column_list = parse_list(params["column_list"])
pk_cols = parse_list(params["pk_columns"])
merge_flag = params["merge_flag"].lower() in ("true","1","yes","y")
incremental_flag = params["incremental_flag"].lower() in ("true","1","yes","y")
modified_col = params["year_column"] if params["year_column"] else None

# ---------------------------
# Configure storage (set spark.conf for ABFSS)
# ---------------------------
STORAGE_ACCOUNT = "scrgvkrmade"
raw_key = params["direct_account_key"]
if raw_key:
    if (raw_key.startswith('"') and raw_key.endswith('"')) or (raw_key.startswith("'") and raw_key.endswith("'")):
        raw_key = raw_key[1:-1]
    raw_key = raw_key.strip()
    # basic validation (heuristic)
    if not raw_key or len(raw_key) < 20:
        raise Exception("direct_account_key looks invalid.")
    spark.conf.set(f"fs.azure.account.key.{STORAGE_ACCOUNT}.dfs.core.windows.net", raw_key)
    print("Spark configured for ABFSS on", STORAGE_ACCOUNT)
else:
    print("No direct_account_key provided — ensure cluster has access to storage.")

# ---------------------------
# Resolve source & target paths (use as-is if full abfss provided)
# ---------------------------
source_path = params["Source_path"]
target_path = params["Target_path"]
if not source_path.lower().startswith("abfss://"):
    raise Exception("Source_path must be full abfss path to bronze table root (e.g. .../Sales.Currency/).")
if not target_path.lower().startswith("abfss://"):
    raise Exception("Target_path must be full abfss path to silver target (e.g. .../silver/stg_currency/).")

# ---------------------------
# Read parquet recursively (all year folders)
# ---------------------------
print("Reading parquet recursively from bronze source:", source_path)
try:
    src_df = (spark.read
              .option("mergeSchema","true")
              .option("recursiveFileLookup","true")
              .parquet(source_path))
    print("Read rows:", src_df.count(), "columns:", src_df.columns)
    display(src_df.limit(5))
except Exception as e:
    traceback.print_exc()
    raise Exception("Failed to read parquet from bronze source: " + str(e))

# ---------------------------
# Keep only required columns (if column_list provided)
# ---------------------------
if column_list:
    # intersection preserves order from column_list
    select_cols = [c for c in column_list if c in src_df.columns]
    missing = [c for c in column_list if c not in src_df.columns]
    if missing:
        print("Warning - these requested columns are missing in bronze source and will be omitted:", missing)
    if not select_cols:
        raise Exception("None of the requested columns exist in source.")
    df = src_df.select(*select_cols)
else:
    df = src_df

print("Selected columns:", df.columns)

# ---------------------------
# Transformations & cleaning
# - Trim string fields
# - Normalize CurrencyCode to upper case
# - Cast ModifiedDate to timestamp
# - Fill / handle nulls where sensible
# - Drop exact duplicate rows by PK (if PK exists)
# ---------------------------
from pyspark.sql.functions import upper, coalesce

# Trim all string columns
string_cols = [c for c,dtype in df.dtypes if dtype == 'string']
for c in string_cols:
    df = df.withColumn(c, trim(col(c)))

# Normalize CurrencyCode (if present)
if "CurrencyCode" in df.columns:
    df = df.withColumn("CurrencyCode", upper(col("CurrencyCode")))

# Convert ModifiedDate to timestamp (if present)
if modified_col and modified_col in df.columns:
    df = df.withColumn(modified_col, to_timestamp(col(modified_col)))
    print("Casted", modified_col, "to timestamp.")

# Add _year column derived from ModifiedDate if possible else try existing _year
if modified_col and modified_col in df.columns:
    df = df.withColumn("_year", year(col(modified_col)))
else:
    # try to detect _year from path or existing column; fallback to literal year
    if "_year" not in df.columns:
        import datetime
        df = df.withColumn("_year", lit(datetime.datetime.utcnow().year))

# Fill nulls for non-key columns where appropriate (simple approach)
# For example, CurrencyName -> empty string if null
if "CurrencyName" in df.columns:
    df = df.withColumn("CurrencyName", coalesce(col("CurrencyName"), lit("")))

# Drop duplicates based on pk_cols if provided
if pk_cols:
    existing_pks = [p for p in pk_cols if p in df.columns]
    if existing_pks:
        before = df.count()
        df = df.dropDuplicates(existing_pks)
        after = df.count()
        print(f"Dropped duplicates by PK {existing_pks}: {before - after} rows removed.")
    else:
        print("PK columns provided but none found in source columns:", pk_cols)

# ---------------------------
# Ensure audit columns exist; do not overwrite if present
# Add/maintain:
#   __ingest_ts (if missing) -> current_timestamp
#   __source_file (if missing) -> blank or derive from _source_file if present
#   __source_path (if missing) -> source_path
#   __batch_id (if missing) -> batch_name from widget
# ---------------------------
if "__ingest_ts" not in df.columns:
    df = df.withColumn("__ingest_ts", current_timestamp())
if "__source_file" not in df.columns:
    # try to derive from file_name widget or leave empty
    _srcfile = params["file_name"] if params["file_name"] else ""
    df = df.withColumn("__source_file", lit(_srcfile))
if "__source_path" not in df.columns:
    df = df.withColumn("__source_path", lit(source_path))
if "__batch_id" not in df.columns:
    df = df.withColumn("__batch_id", lit(params["batch_name"] if params["batch_name"] else ""))

# ---------------------------
# Compute deterministic row hash __row_hash (if not present) using sha2 on concat of PK(s) + all columns
# ---------------------------
# Build concatenation expression: prefer PK columns then all columns
cols_for_hash = []
if pk_cols:
    cols_for_hash = [c for c in pk_cols if c in df.columns]
# fallback to all columns
if not cols_for_hash:
    cols_for_hash = df.columns

# create concatenated string of selected columns
concat_cols_expr = concat_ws("||", *[col(c).cast("string") for c in cols_for_hash])
df = df.withColumn("__row_hash", sha2(concat_cols_expr, 256))

print("After transformations. Columns now:", df.columns)
display(df.limit(5))

# ---------------------------
# MERGE into Delta Silver target (partition by _year)
# ---------------------------
from delta.tables import DeltaTable
import os

# Ensure target_path exists or will be created
print("Preparing to write/merge to target path:", target_path)

# If merge_flag true -> perform MERGE using pk_cols
if merge_flag:
    if not pk_cols:
        raise Exception("merge_flag=true requires pk_columns to be provided.")
    # verify pk exist in df
    pk_in_df = [p for p in pk_cols if p in df.columns]
    if not pk_in_df:
        raise Exception("None of the provided pk_columns exist in the dataframe. pk_columns: " + str(pk_cols))
    # check if delta exists at target_path by trying to read
    delta_exists = False
    try:
        _ = spark.read.format("delta").load(target_path)
        delta_exists = True
    except Exception:
        delta_exists = False

    if not delta_exists:
        # initial write - partition by _year
        print("Target delta does not exist — creating initial Delta at target path (partitioned by _year).")
        (df.write
           .format("delta")
           .mode("overwrite")
           .option("overwriteSchema", "true")
           .partitionBy("_year")
           .save(target_path))
        print("Initial Delta created at", target_path)
    else:
        # perform MERGE
        print("Target delta exists — performing MERGE.")
        try:
            dt = DeltaTable.forPath(spark, target_path)
            src_temp = "_src_df_for_merge_" + uuid.uuid4().hex[:8]
            df.createOrReplaceTempView(src_temp)

            # Build SQL merge strings
            pk_on = " AND ".join([f"target.`{k}` = source.`{k}`" for k in pk_in_df])
            update_cols = [c for c in df.columns if c not in pk_in_df]
            # build update set clause
            update_set = ", ".join([f"target.`{c}` = source.`{c}`" for c in update_cols])
            insert_cols = ", ".join([f"`{c}`" for c in df.columns])
            insert_values = ", ".join([f"source.`{c}`" for c in df.columns])

            merge_sql = f"""
              MERGE INTO delta.`{target_path}` AS target
              USING (SELECT * FROM {src_temp}) AS source
              ON {pk_on}
              WHEN MATCHED THEN
                UPDATE SET {update_set}
              WHEN NOT MATCHED THEN
                INSERT ({insert_cols}) VALUES ({insert_values})
            """
            print("Running MERGE SQL...")
            spark.sql(merge_sql)
            print("MERGE completed successfully.")
        except Exception:
            print("MERGE via SQL failed; attempting DeltaTable API fallback.")
            try:
                dt = DeltaTable.forPath(spark, target_path)
                dt.alias("target").merge(df.alias("source"), pk_on) \
                  .whenMatchedUpdateAll() \
                  .whenNotMatchedInsertAll() \
                  .execute()
                print("MERGE via DeltaTable API completed.")
            except Exception:
                traceback.print_exc()
                raise
else:
    # merge_flag false -> simple write; overwrite for full load, append for incremental
    if incremental_flag:
        write_mode = "append"
    else:
        write_mode = "overwrite"

    (df.write
       .format("delta")
       .mode(write_mode)
       .option("overwriteSchema", "true" if write_mode == "overwrite" else "false")
       .partitionBy("_year")
       .save(target_path))
    print(f"Wrote data to {target_path} with mode {write_mode}.")

# ---------------------------
# Done - show sample from target
# ---------------------------
print("Validating target (reading back small sample):")
try:
    tgt = spark.read.format("delta").load(target_path)
    print("Target row count (approx):", tgt.count())
    display(tgt.limit(5))
except Exception:
    print("Failed to read back Delta; listing target path instead:")
    try:
        for f in dbutils.fs.ls(target_path):
            print(" -", f.path)
    except Exception as e:
        print("Also failed to list target path:", e)

print("04_BronzeToSilver completed.")


In [0]:
# Robust end-to-end cell (no defs). Validates Source_path and avoids "Path must be absolute" error.
import json, traceback, datetime
from pyspark.sql.functions import col, trim, to_timestamp, year, lit, concat_ws, sha2, current_timestamp, coalesce
from delta.tables import DeltaTable

# ---------------------------
# Widgets (ADF will pass these)
# ---------------------------
dbutils.widgets.text("domain", "")
dbutils.widgets.text("file_name", "")            # e.g. Sales.Currency or Sales.Currency.csv
dbutils.widgets.text("column_list", "")          # JSON array OR CSV (not used here)
dbutils.widgets.text("year_column", "")          # optional
dbutils.widgets.text("table_name", "")           # optional override
dbutils.widgets.text("batch_name", "")           # optional batch id/name from ADF
dbutils.widgets.text("direct_account_key", "")   # optional storage account key (base64)
dbutils.widgets.text("Source_path", "")          # optional full path (preferred)
dbutils.widgets.text("Target_path", "")
dbutils.widgets.text("include_layer", "false")   # optional (true/false)
dbutils.widgets.text("layer", "Bronze")          # optional layer name
dbutils.widgets.text("merge_flag", "true")
dbutils.widgets.text("incremental_flag", "false")

# ---------------------------
# Read widget values
# ---------------------------
domain = dbutils.widgets.get("domain").strip()
file_name = dbutils.widgets.get("file_name").strip()
year_column = dbutils.widgets.get("year_column").strip()
table_name = dbutils.widgets.get("table_name").strip()
batch_name = dbutils.widgets.get("batch_name").strip()
direct_account_key = dbutils.widgets.get("direct_account_key").strip()
Source_path = dbutils.widgets.get("Source_path").strip()
Target_path = dbutils.widgets.get("Target_path").strip()
include_layer = dbutils.widgets.get("include_layer").strip().lower() in ("true","1","yes","y")
layer = dbutils.widgets.get("layer").strip()
merge_flag = dbutils.widgets.get("merge_flag").strip().lower() in ("true","1","yes","y")
incremental_flag = dbutils.widgets.get("incremental_flag").strip().lower() in ("true","1","yes","y")

print("Raw widget summary:")
print(" domain:", domain, " file_name:", file_name, " Source_path:", Source_path)
print(" Target_path:", Target_path, " include_layer:", include_layer, " layer:", layer)
print(" merge_flag:", merge_flag, " incremental:", incremental_flag)

# ---------------------------
# Helper: check absolute path
# ---------------------------
def is_absolute_path(p):
    if not p or not isinstance(p, str):
        return False
    low = p.lower()
    return low.startswith("abfss://") or low.startswith("wasbs://") or low.startswith("/dbfs/") or low.startswith("adl://") or low.startswith("s3://")

# ---------------------------
# If user supplied direct_account_key, configure both blob + dfs endpoints
# ---------------------------
# Best-effort extract account name from Target_path or Source_path; fallback to scrgvkrmade
storage_account = None
for p in (Source_path, Target_path):
    if p and "@" in p and ("dfs.core.windows.net" in p or "blob.core.windows.net" in p):
        try:
            storage_account = p.split("@",1)[1].split(".")[0]
            break
        except:
            pass
if not storage_account:
    # fallback to the account we have used in examples
    storage_account = "scrgvkrmade"

if direct_account_key:
    # clean key (strip quotes/newlines)
    k = direct_account_key.strip()
    if (k.startswith('"') and k.endswith('"')) or (k.startswith("'") and k.endswith("'")):
        k = k[1:-1]
    k = k.strip()
    print("Setting storage account keys for account:", storage_account)
    spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", k)
    spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", k)
else:
    print("No direct_account_key provided — ensure cluster identity/secret scope/mount exists for storage access.")

# ---------------------------
# Build Source_path if not absolute (best-effort)
# ---------------------------
if not is_absolute_path(Source_path):
    # If Source_path was empty or wrong, attempt to construct using known pattern:
    # abfss://project@<account>.dfs.core.windows.net/<layer>/<domain>/<file_name>/
    container = "project"
    acct = storage_account
    # layer include
    layer_part = layer if include_layer else ""
    # prefer table_name if provided else file_name (strip extension)
    target_folder = table_name if table_name else file_name
    # try to sanitize
    if target_folder:
        # remove leading slashes
        target_folder = target_folder.strip("/ ")
    else:
        # if no file_name/domain provided, fail with instructive message
        msg = ("Source_path not provided and cannot auto-build path because "
               "both file_name and domain are empty. Please pass widget Source_path "
               "or set domain/file_name. Example:\n"
               "abfss://project@{acct}.dfs.core.windows.net/bronze/<domain>/<file_name>/").format(acct=acct)
        raise RuntimeError(msg)
    # assemble
    # assume Bronze layer root unless layer param differs
    root_layer = layer if layer else "Bronze"
    Source_path = f"abfss://{container}@{acct}.dfs.core.windows.net/{root_layer}/{domain}/{target_folder}/"
    print("Auto-built Source_path ->", Source_path)
else:
    print("Using provided absolute Source_path ->", Source_path)

# final validate
if not is_absolute_path(Source_path):
    raise RuntimeError("Final Source_path is not an absolute path: " + str(Source_path))

# ---------------------------
# READ SOURCE parquet (recursive)
# ---------------------------
try:
    print("Reading parquet data from ->", Source_path)
    src_df = (spark.read
              .option("mergeSchema","true")
              .option("recursiveFileLookup","true")
              .parquet(Source_path))
    print("Read OK. Columns:", src_df.columns)
    display(src_df.limit(5))
except Exception:
    traceback.print_exc()
    raise RuntimeError("Failed to read parquet from Source_path: " + str(Source_path))

# ---------------------------
# CLEAN / TRANSFORM using ALL available columns (generic)
# ---------------------------
df = src_df
# trim strings
string_cols = [c for c,d in df.dtypes if d == "string"]
for c in string_cols:
    df = df.withColumn(c, trim(col(c)))
# year -> timestamp and _year if available
if year_column and year_column in df.columns:
    try:
        df = df.withColumn(year_column, to_timestamp(col(year_column)))
        df = df.withColumn("_year", year(col(year_column)))
    except Exception:
        if "_year" not in df.columns:
            df = df.withColumn("_year", lit(datetime.datetime.utcnow().year))
else:
    if "_year" not in df.columns:
        df = df.withColumn("_year", lit(datetime.datetime.utcnow().year))
# audit columns
if "__ingest_ts" not in df.columns:
    df = df.withColumn("__ingest_ts", current_timestamp())
if "__source_file" not in df.columns:
    df = df.withColumn("__source_file", lit(Source_path.split("/")[-1] or Source_path))
if "__source_path" not in df.columns:
    df = df.withColumn("__source_path", lit(Source_path))
if "__batch_id" not in df.columns:
    dbatch = batch_name if batch_name else "Batch-" + datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S")
    df = df.withColumn("__batch_id", lit(dbatch))

# compute generic __row_hash using all columns
concat_expr = concat_ws("||", *[col(c).cast("string") for c in df.columns])
df = df.withColumn("__row_hash", sha2(concat_expr, 256))

print("After generic cleaning. Columns now:", df.columns)
display(df.limit(5))

# ---------------------------
# DEDUPE (use pk if provided via pk_columns in column_list widget or fallback to __row_hash)
# ---------------------------
# try to read pk_columns from column_list if user passed json and included pk info (best-effort)
pk_list = []
try:
    maybe_cols = dbutils.widgets.get("column_list").strip()
    if maybe_cols:
        # try json parse
        try:
            parsed = json.loads(maybe_cols)
            if isinstance(parsed, dict) and parsed.get("pk_columns"):
                pk_list = parsed.get("pk_columns", [])
            elif isinstance(parsed, list):
                # nothing special; user passed columns list; no pk
                pk_list = []
        except Exception:
            pk_list = []
except Exception:
    pk_list = []

if pk_list:
    valid_pk = [p for p in pk_list if p in df.columns]
else:
    valid_pk = []

if valid_pk:
    try:
        before = df.count()
    except Exception:
        before = None
    df = df.dropDuplicates(valid_pk)
    if before is not None:
        after = df.count()
        print(f"Dropped duplicates by PK {valid_pk}: {before-after} rows removed.")
else:
    # fallback: dedupe by __row_hash
    df = df.dropDuplicates(["__row_hash"])
    print("Dropped duplicates by __row_hash.")

# ---------------------------
# WRITE / MERGE into Delta
# ---------------------------
print("Preparing to write/merge to target Delta:", Target_path)
delta_exists = True
try:
    _ = spark.read.format("delta").load(Target_path)
except Exception:
    delta_exists = False

if not delta_exists:
    print("Target Delta not found -> creating initial Delta.")
    (df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .partitionBy("_year")
        .save(Target_path))
    print("Initial Delta created at:", Target_path)
else:
    # choose merge key: try CurrencyAlternateKey if present, else __row_hash
    merge_pks = [c for c in ["CurrencyAlternateKey"] if c in df.columns]
    if not merge_pks:
        merge_pks = ["__row_hash"]
    join_cond = " AND ".join([f"target.`{c}` = source.`{c}`" for c in merge_pks])
    print("Merging using keys:", merge_pks)
    dt = DeltaTable.forPath(spark, Target_path)
    dt.alias("target").merge(
        df.alias("source"),
        join_cond
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()
    print("MERGE completed into:", Target_path)

# ---------------------------
# final sample
# ---------------------------
try:
    tgt = spark.read.format("delta").load(Target_path)
    print("Target approx row count:", tgt.count())
    display(tgt.limit(5))
except Exception as e:
    print("Could not read target after write/merge:", e)
    try:
        for f in dbutils.fs.ls(Target_path):
            print("-", f.path)
    except Exception as e2:
        print("Also failed to list target path:", e2)

print("End.")
